In [1]:
using LinearAlgebra
using ForwardDiff
using SatelliteDynamics
using DifferentialEquations
using Plots
using ECOS, Convex
using Mosek, MosekTools

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


In [2]:
#need to find a way to sample the larger part of the trajectory at a different frequency...

#because currently the amount of knot points is huge...

In [3]:
#From RoadsMPC slide deck
#values are in m and degrees 
#orbits elements of und roads 1
#last is the true anomaly 
elements1 = [6885.635e3, 0.001089, 97.43, 0, 0, 0]

#orbital elements of und roads 2
#last is the true anomaly
elements2 = [6885.635e3, 0.001089, 97.43, 0, 0, 0.016667]

6-element Vector{Float64}:
  6.885635e6
  0.001089
 97.43
  0.0
  0.0
  0.016667

In [4]:
#convert the true anomaly to mean anomaly to be able to do the conversion and degrees to radians

In [5]:
function degrees_to_rad(x)

    return x*pi/180
    
end

degrees_to_rad (generic function with 1 method)

In [6]:
#v is true anomaly 
#e is the eccentricity
function true_anom_to_eccentric_anom(v, e)

    E = 2*atand(sqrt((1-e)/(1+e))*tand(v/2))

    #eccentric anomaly
    return E

end

true_anom_to_eccentric_anom (generic function with 1 method)

In [7]:
function eccentric_anom_to_mean_anom(E, e)

    M = E - e*sind(E)

    #mean anomaly
    return M

end

eccentric_anom_to_mean_anom (generic function with 1 method)

In [8]:
#eccentric anomaly sat 1
E1 = true_anom_to_eccentric_anom(elements1[end], elements1[2])

#mean anomaly sat 1
M1 = eccentric_anom_to_mean_anom(E1, elements1[2])

0.0

In [9]:
#eccentric anomaly sat 2
E2 = true_anom_to_eccentric_anom(elements2[end], elements2[2])

#eccentric anomaly sat 2
M2  = eccentric_anom_to_mean_anom(E2, elements2[2])

0.016648543070573774

In [10]:
#get all the units consistent (m, radians)

elements1_m = [elements1[1:2]; degrees_to_rad(elements1[3]); degrees_to_rad(elements1[4]); degrees_to_rad(elements1[5]); degrees_to_rad(M1)]

elements2_m = [elements2[1:2]; degrees_to_rad(elements2[3]); degrees_to_rad(elements2[4]); degrees_to_rad(elements2[5]); degrees_to_rad(M2)]

6-element Vector{Float64}:
 6.885635e6
 0.001089
 1.7004742902180754
 0.0
 0.0
 0.00029057189224159907

In [11]:
#state satellite 1 (target)

x_1 = sOSCtoCART(elements1_m, use_degrees=false)

6-element Vector{Float64}:
    6.878136543485e6
    0.0
    0.0
    0.0
 -984.9589353739226
 7552.79897887146

In [12]:
#state satellite 2 (chaser)

x_2 = sOSCtoCART(elements2_m, use_degrees=false)

6-element Vector{Float64}:
    6.878136252166805e6
 -259.0113696278884
 1986.134383662354
   -2.2156282462577286
 -984.9588936567426
 7552.798658978454

In [13]:
#relative state of chaser wrt target
#this is in ECI frame
difference = x_2 - x_1 

6-element Vector{Float64}:
   -0.2913181949406862
 -259.0113696278884
 1986.134383662354
   -2.2156282462577286
    4.171718001089175e-5
   -0.0003198930062353611

In [14]:
#this is in the RTN frame of the target (primary observing satellite) 
initial_difference = sECItoRTN(x_1, x_2) 

6-element StaticArraysCore.SVector{6, Float64} with indices SOneTo(6):
   -0.2913181949406862
 2002.9519913273894
    2.7017514072632172e-14
    0.002412819262479271
   -1.5962127995240716e-12
    3.511794450825904e-14

In [15]:
#unit scaling to make the numerics better (no overshoot)
position_scale = 100
time_scale = 1000

velocity_scale = position_scale/time_scale
acceleration_scale = position_scale/(time_scale)^2

0.0001

In [16]:
#mu - gravitational parameter (m3/s2)
mu = 3.986e14

3.986e14

In [17]:
mu_scaled = mu/(position_scale^3/time_scale^2)

3.986e14

In [18]:
#target orbit period in seconds
T = 2*pi*sqrt((elements1[1]^3)/(mu))

5686.266596687438

In [19]:
#semi major axis of target satellite (meters)
a = elements1[1]

6.885635e6

In [20]:
#scaled semi major axis
a_scaled = a/position_scale

68856.35

In [21]:
#constant mean motion of target n = sqrt(mu/a^3)
#units: rad/s
n = sqrt(mu/a^3)

0.0011049755055170796

In [22]:
n_scaled = sqrt(mu_scaled/a_scaled^3)

1.1049755055170793

In [23]:
#number of states
nx = 6

#number of control inputs
nu = 3

3

In [24]:
#timestep for first 120 steps
dt_120 = 1

1

In [25]:
dt_120_scaled = dt_120/time_scale 

0.001

In [26]:
dt_92 = 60

60

In [27]:
dt_92_scaled = dt_92/time_scale

0.06

In [28]:
#when sampling at 1 second
#N = 94*60+1

In [29]:
# #this is a 5 orbits for dt=60

# #one orbit 
N_120 = 181
N_92 = 91

total_N = N_120 + N_92

# total_N = 212
# #2 orbit look ahead? 
# #N = 188

272

In [30]:
x0_target = zeros(nx) 

6-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [31]:
#this is in RTN frame
# m and s
x0_chaser = initial_difference

6-element StaticArraysCore.SVector{6, Float64} with indices SOneTo(6):
   -0.2913181949406862
 2002.9519913273894
    2.7017514072632172e-14
    0.002412819262479271
   -1.5962127995240716e-12
    3.511794450825904e-14

In [32]:
A = zeros(nx, nx)
A[1:3, 4:6] = I(3)
A[4, 1] = 3*n^2
A[6, 3] = -n^2
A[4, 5] = 2*n
A[5, 4] = -2*n

-0.002209951011034159

In [33]:
A_scaled = zeros(nx, nx)
A_scaled[1:3, 4:6] = I(3)
A_scaled[4, 1] = 3*n_scaled^2
A_scaled[6, 3] = -n_scaled^2
A_scaled[4, 5] = 2*n_scaled
A_scaled[5, 4] = -2*n_scaled

-2.2099510110341587

In [34]:
#mass of the satellite (kg) from data
m = 5.22

5.22

In [35]:
#in m/s2. L1 norm 
umax = 4.6e-3

0.0046

In [36]:
umax_scaled = umax/acceleration_scale 

46.0

In [37]:
#si units
B = [zeros(3,3); I(3)]

6×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [38]:
#continous dynamics
function spacecraft_dynamics(x,u)

    xdot = A*x + B*u

    return xdot

end

spacecraft_dynamics (generic function with 1 method)

In [39]:
dt_120_scaled 

0.001

In [40]:
dt_92_scaled

0.06

In [41]:
#discretize the dynamics model 
#for 1 Hz
H_scaled_120 = exp(dt_120_scaled*[A_scaled B; zeros(nu, nx+nu)])

9×9 Matrix{Float64}:
  1.0         0.0   0.0         …   5.0e-7       3.68325e-10  0.0
 -1.34914e-9  1.0   0.0            -3.68325e-10  5.0e-7       0.0
  0.0         0.0   0.999999        0.0          0.0          5.0e-7
  0.00366291  0.0   0.0             0.001        1.10498e-6   0.0
 -4.04743e-6  0.0   0.0            -1.10498e-6   0.000999999  0.0
  0.0         0.0  -0.00122097  …   0.0          0.0          0.001
  0.0         0.0   0.0             1.0          0.0          0.0
  0.0         0.0   0.0             0.0          1.0          0.0
  0.0         0.0   0.0             0.0          0.0          1.0

In [42]:
#discretize the dynamics model 
#for dt=60 s ~ 0.017 Hz
H_scaled_92 = exp(dt_92_scaled*[A_scaled B; zeros(nu, nx+nu)])

9×9 Matrix{Float64}:
  1.00659      0.0   0.0        …   0.00179934  7.95408e-5  0.0
 -0.000291351  1.0   0.0           -7.95408e-5  0.00179736  0.0
  0.0          0.0   0.997803       0.0         0.0         0.00179934
  0.219614     0.0   0.0            0.0599561   0.00397645  0.0
 -0.0145654    0.0   0.0           -0.00397645  0.0598242   0.0
  0.0          0.0  -0.0732046  …   0.0         0.0         0.0599561
  0.0          0.0   0.0            1.0         0.0         0.0
  0.0          0.0   0.0            0.0         1.0         0.0
  0.0          0.0   0.0            0.0         0.0         1.0

In [43]:
#Scaled Discrete Dynamics Matrices for dt = 1 s
Ad_scaled_120  = H_scaled_120[1:nx, 1:nx]
Bd_scaled_120 = H_scaled_120[1:nx, (nx+1):end]

6×3 Matrix{Float64}:
  5.0e-7       3.68325e-10  0.0
 -3.68325e-10  5.0e-7       0.0
  0.0          0.0          5.0e-7
  0.001        1.10498e-6   0.0
 -1.10498e-6   0.000999999  0.0
  0.0          0.0          0.001

In [44]:
#Scaled Discrete Dynamics Matrices for dt= 60 s
Ad_scaled_92 = H_scaled_92[1:nx, 1:nx]
Bd_scaled_92 = H_scaled_92[1:nx, (nx+1):end]

6×3 Matrix{Float64}:
  0.00179934  7.95408e-5  0.0
 -7.95408e-5  0.00179736  0.0
  0.0         0.0         0.00179934
  0.0599561   0.00397645  0.0
 -0.00397645  0.0598242   0.0
  0.0         0.0         0.0599561

In [45]:
function spacecraft_dynamics_discrete_scaled(x_scaled, u_scaled, Ad_scaled, Bd_scaled)

    xnext_scaled = Ad_scaled*x_scaled + Bd_scaled*u_scaled

    return xnext_scaled

end

spacecraft_dynamics_discrete_scaled (generic function with 1 method)

In [46]:
#simulate for 1 minute


#x scaled will be a vector

#u scaled will be a control trajectory (3x60)


function sim_one_min(x_scaled, u_traj, Ad_scaled_120, Bd_scaled_120)

    all_x = zeros(6, 61)
    all_x[:,1] = x_scaled

    for i=1:size(u_traj)[2]

        all_x[:,i+1] = spacecraft_dynamics_discrete_scaled(all_x[:,i], u_traj[:,i], Ad_scaled_120, Bd_scaled_120)

    end

    xnext = all_x[:,end]

    return xnext

end

sim_one_min (generic function with 1 method)

In [47]:
x_initial = x0_chaser

6-element StaticArraysCore.SVector{6, Float64} with indices SOneTo(6):
   -0.2913181949406862
 2002.9519913273894
    2.7017514072632172e-14
    0.002412819262479271
   -1.5962127995240716e-12
    3.511794450825904e-14

In [48]:
x_initial_scaled = [x0_chaser[1:3]/position_scale; x0_chaser[4:6]/velocity_scale] 

6-element Vector{Float64}:
 -0.002913181949406862
 20.029519913273894
  2.701751407263217e-16
  0.02412819262479271
 -1.5962127995240716e-11
  3.511794450825904e-13

In [49]:
x_goal = x0_target

6-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [50]:
C = zeros(N_120-1-59, N_120-1)

121×180 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱            ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

In [51]:
for i=1:size(C)[1]

    C[i, i:i+59] = ones(60)
    
end

In [52]:
C

121×180 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱            ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

In [53]:
function update_prob(x_initial_k, C_k)


    X = Convex.Variable(nx, total_N)
    U = Convex.Variable(nu, total_N-1)
    
    #Initial State Constraint
    cons = Constraint[X[:,1] == x_initial_k]

    #Dynamics Constraint
    # for k=1:(N-1)

    #     push!(cons, X[:,k+1] - (Ad_scaled_120*X[:,k] + Bd_scaled_120*U[:,k]) == zeros(6))

    # end



    #for the first 2 minutes sampled at 1 Hz
    for k=1:N_120-1

        push!(cons, X[:,k+1] - (Ad_scaled_120*X[:,k] + Bd_scaled_120*U[:,k]) == zeros(6))

    end

    #for the remaining of the trajectory 
    for k=N_120:total_N - 1

        push!(cons, X[:,k+1] - (Ad_scaled_92*X[:,k] + Bd_scaled_92*U[:,k]) == zeros(6))

    end


    # for k=1:60

    #     push!(cons, sum(U[:,k:k+59]) <= umax/acceleration_scale)
        
    # end


    
    #control constraint for first 2 minutes
    push!(cons, C_k*([1 1 1]*abs(U[:, 1:N_120-1]))' <= umax/acceleration_scale*ones(size(C)[1]))


    #add in thrust constraints

    #try both l1 and l2

    # for k=N_2_min:N_2_min + N_92_min - 1

    #    #L2 thrust constraint
    #    #push!(cons, norm(U[:,k],2) <= umax/acceleration_scale)

    # #    #L1 thrust constraint
    # #works when bound is 1

    #push!(cons, norm(U[:,k],1) <= (umax/60/acceleration_scale))

    # #push!(cons, U[:,k] == zeros(3))

    # end

    #control constraint (testing)
    for k=N_120:(total_N-1)

       push!(cons, norm(U[:,k], 1) <= umax/60/acceleration_scale)
       

    end


    #slow progress... (kinda working?)
    #the constraint with C gives a similar solution, however, it is faster 
    # for k=1:(N-1)-59

    #     #slow progress
    #     push!(cons, sum(abs(U[:,k:k+59])) <= umax/acceleration_scale)

    # end




    
    #not working
    # for k=1:N-1-59

    #     normz_60 = 0

    #     for j = k:k+59

    #         normz_60 += norm(U[:,j], 1) 

    #     end

    #     push!(cons, normz_60 <= umax/acceleration_scale)

    # end
    

    return cons, X, U

end

update_prob (generic function with 1 method)

In [54]:
function solve_prob(cons, X, U, N)


    #objective 
    obj = 0

    #weights depending how conservative you want to be on fuel vs reaching the goal
    #sensative to these weights. with beta as 100 it wouldn't solve...

    #these weights do something (need tuning)
    # beta_120 = 0.1
    # alpha_120 = 100

    # beta_92 = 100
    # alpha_92 = 1
    # alpha_92_end = 1000

    #works kinda (converges to zero kinda)
    # beta_120 = 0.1
    # alpha_120 = 1

    # beta_92 = 100
    # alpha_92 = 1
    # alpha_92_end = 1


    #these work the best
    beta_120 = 0.1
    alpha_120 = 1

    beta_92 = 1
    alpha_92 = 100
    alpha_92_end = 100


    

    #testing (not working)
    # beta_120 = 1
    # alpha_120 = 1

    # beta_92 = 1000
    # alpha_92 = 1
    # alpha_92_end = 1

    #trying old weights (the satellite finds it optimal to just do nothing...)
    # beta_120 = 1
    # alpha_120 = 1

    # beta_92 = 1
    # alpha_92 = 1
    # alpha_92_end = 1

    #Initial 2 minute trajectory
    for k=1:N_120-1

        obj += beta_120*norm(U[:,k], 1) + alpha_120*norm(X[:,k], 2)

    end

    #Rest of the trajectory
    for k=N_120:total_N - 1

        obj += beta_92*norm(U[:,k], 1) + alpha_92*norm(X[:,k], 2)

    end

    #terminal cost
    obj += alpha_92_end*norm(X[:,N], 2)



    #working previous cost (all trajectory)
    # for k=1:total_N-1
    #     #trying L2 cost on controls (doesn't work, all zeros)
        
    #     #L1 gets a solution but it is not completely correct...
    #     obj += beta*norm(U[:,k], 1) + alpha*norm(X[:,k], 2)

    # end

    # #terminal cost
    # obj += alpha*norm(X[:,total_N], 2)

    prob = minimize(obj, cons)

    #solve with ECOS
    #Convex.solve!(prob, ()-> ECOS.Optimizer(), silent_solver=true);

    Convex.solve!(prob, ()-> Mosek.Optimizer(), silent_solver = true)

    Xm = X.value
    Um = U.value

    return Xm, Um

end

solve_prob (generic function with 1 method)

In [55]:
#iterations = 1000
num_orbits = 3

#iterations = N*num_orbits/60

iterations = 400

400

In [56]:
#set up in Convex.jl

In [57]:
xhist = zeros(nx, iterations)
uhist = zeros(nu, (iterations-1)*60)

xhist[:,1] = x_initial_scaled

6-element Vector{Float64}:
 -0.002913181949406862
 20.029519913273894
  2.701751407263217e-16
  0.02412819262479271
 -1.5962127995240716e-11
  3.511794450825904e-13

In [58]:
C_k = C

121×180 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱            ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

In [59]:
uhist_all = zeros(3, 60, iterations-1)

3×60×399 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

;;; … 

[:, :, 397] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0 

In [60]:
all_C_k = zeros(121, 180, iterations)

121×180×400 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱            ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [61]:
for i=1:iterations-1

#for i=1:iterations-1

#for 60 second constraint
#for i in iters_odd 

#took 50 minutes for 10 iterations. output was just zero controls...
#for i=1:10

# i=1

    all_C_k[:,:, i] = C_k
    
    cons, X, U = update_prob(xhist[:,i], C_k)

    Xm, Um = solve_prob(cons, X, U, total_N)

    #take one step 
    #uhist[:,i] = Um[:,1]

    u_traj = Um[:,1:60]

    uhist_all[:,:, i] = u_traj

    uhist[:, (i-1)*60+1: (i-1)*60+60] = Um[:,1:60]

    max_sum = vec(sum(abs.(u_traj), dims=1))

    control_index = findmax(max_sum)[2]

    println("this is control index: ", control_index)
    
    #modify the C to repect the 60 second constraint
    #commenting out this logic 
    if control_index ==1

        C_k = C

    else
        #println("this is control index: ", control_index)
        C_k = C

        #changing rows
        #C_k[1:control_index-1, :] = zeros(control_index-1, N_120-1)

        #changing columns
        C_k[:, 1:control_index-1] = zeros( 121 ,control_index-1)

    end

    #uhist[:,i+1] = Um[:,2]

    xhist[:,i+1] = sim_one_min(xhist[:,i], u_traj, Ad_scaled_120, Bd_scaled_120)

    println("Minute: ", i, " done")

    #xhist[:,i+1] = spacecraft_dynamics_discrete_scaled(xhist[:,i], uhist[:,i])

    #xhist[:,i+2] = spacecraft_dynamics_discrete(xhist[:,i+1], uhist[:,i+1])

end

this is control index: 1
Minute: 

┌ Warning: Problem status SLOW_PROGRESS; solution may be inaccurate.
└ @ Convex /home/faustovega/.julia/packages/Convex/b2S4H/src/solution.jl:342


1 done
this is control index: 1
Minute: 2 done
this is control index: 

┌ Warning: Problem status SLOW_PROGRESS; solution may be inaccurate.
└ @ Convex /home/faustovega/.julia/packages/Convex/b2S4H/src/solution.jl:342


1
Minute: 3 done
this is control index: 1
Minute: 4 done


...

60
Minute: 104 done
this is control index: 60
Minute: 105 done
this is control index: 60
Minute: 106 done
this is control index: 60
Minute: 107 done
this is control index: 

┌ Warning: Problem status SLOW_PROGRESS; solution may be inaccurate.
└ @ Convex /home/faustovega/.julia/packages/Convex/b2S4H/src/solution.jl:342


60
Minute: 108 done
this is control index: 60
Minute: 109 done
this is control index: 1
Minute: 110 done
this is control index: 1
Minute: 111 done
this is control index: 1
Minute: 112 done
this is control index: 1
Minute: 113 done
this is control index: 1
Minute: 114 done
this is control index: 1
Minute: 115 done
this is control index: 1
Minute: 116 done
this is control index: 1
Minute: 117 done
this is control index: 1
Minute: 118 done
this is control index: 1
Minute: 119 done
this is control index: 1
Minute: 120 done
this is control index: 1
Minute: 121 done
this is control index: 1
Minute: 122 done
this is control index: 1
Minute: 123 done
this is control index: 1
Minute: 124 done
this is control index: 1
Minute: 125 done
this is control index: 1
Minute: 126 done
this is control index: 1
Minute: 127 done
this is control index: 1
Minute: 128 done
this is control index: 1
Minute: 129 done
this is control index: 1
Minute: 130 done
this is control index: 1
Minute: 131 done
this is contr

┌ Warning: Problem status SLOW_PROGRESS; solution may be inaccurate.
└ @ Convex /home/faustovega/.julia/packages/Convex/b2S4H/src/solution.jl:342


3
Minute: 270 done
this is control index: 2
Minute: 271 done
this is control index: 1
Minute: 272 done
this is control index: 1
Minute: 273 done
this is control index: 1
Minute: 274 done
this is control index: 2
Minute: 275 done
this is control index: 2
Minute: 276 done
this is control index: 3
Minute: 277 done
this is control index: 3
Minute: 278 done
this is control index: 1
Minute: 279 done
this is control index: 3
Minute: 280 done
this is control index: 8
Minute: 281 done
this is control index: 2
Minute: 282 done
this is control index: 3
Minute: 283 done
this is control index: 2
Minute: 284 done
this is control index: 1
Minute: 285 done
this is control index: 2
Minute: 286 done
this is control index: 1
Minute: 287 done
this is control index: 1
Minute: 288 done
this is control index: 3
Minute: 289 done
this is control index: 2
Minute: 290 done
this is control index: 1
Minute: 291 done
this is control index: 1
Minute: 292 done
this is control index: 1
Minute: 293 done
this is control

In [ ]:
all_C_k[:,:,109]

In [ ]:
#all_C_k[:,:,109]

In [ ]:
all_C_k[:,:,110]

In [ ]:
all_C_k[1:60,:,110]

In [ ]:
uhist_all[:,:, 109]

In [ ]:
uhist_all[:,:, 110]

In [ ]:
plot(sum(abs.(uhist_all[:,:, 64]), dims=1)')

In [ ]:
plot([uhist_all[:,:,109] uhist_all[:,:, 110]]')

In [ ]:
test = uhist[:, 3600-59:3661]

In [ ]:
plot(test') 

In [ ]:
uhist_all[:,:, 50]

In [ ]:
uhist_all[:,:, 51]

In [ ]:
C_k 

In [ ]:
index = 2
C_k_new = C
C_k_new[1:index-1, :] = zeros(180)

In [ ]:
C_k_new 

In [ ]:
xhist 

In [ ]:
plot(xhist' )

In [ ]:
156*60

In [ ]:
54*60

In [ ]:
55*60

In [ ]:
plot(uhist[:, 3200:3350]')

In [ ]:
uhist[:, 3001]

In [ ]:
uhist[:, 3061]

In [ ]:
uhist[:, 3181]

In [ ]:
plot(uhist[:,1:360]')

In [ ]:
plot(uhist[:, 1:300]')

In [ ]:
xhist_si = [xhist[1:3, :]*position_scale; xhist[4:6, :]*velocity_scale]  

In [ ]:
#rescale uhist into m/s2

uhist_si = uhist*acceleration_scale 

In [ ]:
max_sum = vec(sum(uhist_si[:,1:60], dims=1))

findmax(max_sum)[2]

In [ ]:
plot(xhist_si[1,:], xhist_si[2,:], xhist_si[3,:], label = "Chaser Trajectory", zlim = (-1,1))

scatter!([xhist_si[1,1]], [xhist_si[2,1]], [xhist_si[3,1]], label = "Initial State")
scatter!([xhist_si[1,end]], [xhist_si[2,end]], [xhist_si[3,end]], label = "End State")

scatter!([0], [0], [0], label= "Target")

In [ ]:
plot(xhist_si[2,:], xhist_si[1,:], xlabel= "Tangential (m)", ylabel="Radial (m)")

In [ ]:
plot(xhist_si[3,:], xhist_si[1,:], xlabel= "Normal (m)", ylabel="Radial (m)", label=false)

In [ ]:
plot(xhist_si[1,:], title="Positions") 
plot!(xhist_si[2,:])
plot!(xhist_si[3,:])

In [ ]:
plot(xhist_si[4,:], title="Velocities") 
plot!(xhist_si[5,:])
plot!(xhist_si[6,:])

In [ ]:
uhist_si 

In [ ]:
uhist_si[:, 1:60]

In [ ]:
plot(uhist_si[2, :])

In [ ]:
plot(uhist_si[2,3000:3100])

In [ ]:
uhist_si[:,3000]

In [ ]:
50*60

In [ ]:
plot(uhist_si[1,:], title = "Controls")
plot!(uhist_si[2,:])
plot!(uhist_si[3,:])

In [ ]:
C 

In [ ]:
plot(uhist_si[1,1:10000], title = "Controls", label="u_x", xlabel = "Time (s)", ylabel = "Acceleration (m/s2)")
plot!(uhist_si[2,1:10000], label="u_y")
plot!(uhist_si[3,1:10000], label="u_z")

In [ ]:
plot(uhist_si[1,7000:7500], title = "Controls")
plot!(uhist_si[2,7000:7500])
plot!(uhist_si[3,7000:7500])

In [ ]:
plot(uhist_si[2, 1:360])

In [ ]:
#units: m/s2
fuel_usage_accel = sum(abs.(vec(uhist_si)))

In [ ]:
#these are 1 second impulses
#units: m/s
delta_v = 1*fuel_usage_accel 

In [ ]:
no_control = zeros(nx, iterations)

In [ ]:
no_control[:,1] = x0_chaser

In [ ]:
for i = 1:iterations-1

    no_control[:, i+1] = Ad*no_control[:,i]

end

In [ ]:
# plot(no_control[1,:], no_control[2,:], no_control[3,:], zlims=(-1,1))
# scatter!([0], [0], [0])

In [ ]:
plot(no_control[2,:], no_control[1,:])

In [ ]:
plot(no_control[3,:], no_control[1,:])

In [ ]:
Ad_scaled 